# Data cleaning for movies dataset
**Dataset found from https://www.kaggle.com/rounakbanik/the-movies-dataset/data **

We remove columns that are not needed and encoded categorical variables like genres, region of countries and the season of the release date of the movies.
We only would select and analyse movies released from 2008 onwards.

In [19]:
from pandas import *
import re
import numpy as np 
import scipy as sp 

def extract(x):
    return re.findall(r'name\': \'(\w+)\'',x)

meta=read_csv('/home/angps/Documents/DSC3216 (1)/project/the-movies-dataset/movies_metadata.csv')
meta.drop(['status','belongs_to_collection','homepage','imdb_id','poster_path','spoken_languages','original_title','video'],axis=1,inplace=True)

meta['genres']=meta['genres'].apply(lambda x:extract(x))
genres = set()
for x in meta.genres.values:
    for i in x:
        genres.add(i)

for genre in genres:
    meta[genre] = meta['genres'].apply(lambda list:1 if genre in list else 0)
    
meta['production_companies']=meta['production_companies'].astype(str)
meta['production_companies']=meta['production_companies'].apply(lambda x:extract(x))

meta['production_countries']=meta['production_countries'].astype(str)
meta['production_countries']=meta['production_countries'].apply(lambda x:extract(x))

country=read_excel("/home/angps/Documents/DSC3216 (1)/project/Copy of List of countries - Table 9.xlsm")
country.set_index("Area",inplace=True)

areas = set()
for x in country.index.values:
    areas.add(x)
    
def in_area(country,area):
    for i in country:
        if i in area:
            return True
    return False

for area in areas:
    meta[area] = meta['production_countries'].apply(lambda list:1 if in_area(list, country.loc[area,"Countries"]) else 0)


meta.drop([19730,29503,35587],inplace=True)
meta['id']=meta.id.astype(int,errors='ignore')


def extract2(x,n):
    return re.findall(r'name\': \'(\w+ \w*)\'',x)[:n]

credit = read_csv('/home/angps/Documents/DSC3216 (1)/project/the-movies-dataset/credits.csv')
credit['top_cast'] = credit['cast'].apply(lambda x: extract2(x,2))
credit['top_crew'] = credit['crew'].apply(lambda x: extract2(x,2))
credit.drop(['cast','crew'],axis=1,inplace=True)


meta["is_adult_rated"]=np.where(meta["adult"]=="True",1,0)
meta.drop(["Aniplex","GoHands"],inplace=True,axis=1)

full_data = meta.merge(credit,how='inner',on='id')
data=full_data[full_data["release_date"]>="2008-01-01"]
data.reset_index(inplace=True)


data["month"]=data["release_date"].apply(lambda x: int(x[5:7]))
def season(data): #if spring,summer,autumn is 0, then winter is 1
    data["Spring"]=np.where(((data["month"]>=3) & (data["month"]<=5)),1,0)
    data["Summer"]=np.where(((data["month"]>=6) & (data["month"]<=8)),1,0)
    data["Autumn"]=np.where(((data["month"]>=9) & (data["month"]<=11)),1,0)
season(data)
data.drop(["month","index"],inplace=True,axis=1)
data.drop_duplicates(subset="id",keep="first",inplace=True)
data.reset_index(inplace=True)
#combined.set_index('title',inplace=True)




/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/angps/anacon

In [20]:
full_data.to_csv("full_data.csv",index=False)
data.to_csv("data.csv",index=False)

**We would then select only rows where there are recorded (non-zero) values in budget and revenue to make our data more consistent.**

In [21]:
data=read_csv("/home/angps/Documents/DSC3216 (1)/project/data.csv")
valid=data[data["budget"]>0]
valid_data=valid[valid["revenue"]>0]
valid_data.reset_index(inplace=True,drop=True)
valid_data.to_csv("valid_data.csv",index=False)